# Traitement et modélisation des images
<br>

## Preprocessing

#### Importation des ressources

In [1]:
# Importation des librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray

# Importation du module os pour accéder aux images
import os
path = '../../../images/image_train'
list_img = os.listdir(path)

print('le nombre total d\'image dans le repertoire image_train est de :', len(list_img))

le nombre total d'image dans le repertoire image_train est de : 84916


#### fusion des deux datasets

In [2]:
# Chargement des fichiers X_train et Y_train
X_train = pd.read_csv('../../../X_train.csv', index_col=0)
y_train = pd.read_csv('../../../y_train.csv', index_col=0)

# Fusion avec merge des deux datasets
df = pd.merge(X_train, y_train, left_index = True, right_index = True)

#### Creation d'une nouvelle colonne contenant le chemin pour accéder à l'image

In [3]:
# création d'une nouvelle colonne
df['Nom image'] = ['image_' + str(imageid) + '_product_' + str(productid) + '.jpg' for imageid, productid in zip(df['imageid'], df['productid'])]
df['lien'] = str(path) + '/' + df['Nom image']

#### Prétraitement des images 

In [4]:
# Fonction pour lire et prétraiter une image
def preprocess_image(image_path, target_size=(64, 64)):
    image = imread(image_path)
    image = rgb2gray(image)  # Convertir en noir et blanc
    image = resize(image, target_size)  # Redimensionner l'image
    return image.flatten()  # Aplatir l'image en un vecteur 1D

# Appliquer le prétraitement à toutes les images et stocker les caractéristiques
features = []
for image_lien in df['lien']:
    features.append(preprocess_image(image_lien))

features = np.array(features)

## Modelisation Random Forest

#### Importation des ressources

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import joblib

#### Entrainement

In [11]:
# Préparer les données pour l'entraînement
X = features
y = df['prdtypecode'].values

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configuration de GridSearchCV
"""
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 7],
    'min_samples_leaf': [1, 2, 3]
}
"""
param_grid = {
    'n_estimators': [50, 100, 150]
}

# Initialiser le modèle Random Forest
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Initialiser GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Exécuter GridSearchCV sur les données d'entraînement
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres:", grid_search.best_params_)

# Utiliser le meilleur modèle trouvé par GridSearchCV
best_model = grid_search.best_estimator_

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Meilleurs paramètres: {'n_estimators': 150}


#### Evaluation du modèle 

In [12]:
# Évaluer le modèle
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy du meilleur modèle: {accuracy*100:.2f}%')

Accuracy du meilleur modèle: 47.76%


#### Sauvegarde du model

In [13]:
# Sauvegarder le meilleur modèle
joblib.dump(best_model, 'random_forest_best_model.joblib')

['random_forest_best_model.joblib']